## Lab 06

### Zadanie 1 - Regexy

Przejdź 15 zadań ze strony http://regexone.com/

### Zadanie 2 - Regexy

_Atoms_

Napisz program, który (używając wyrażeń regularnych) przefiltruje plik atoms.log tak by pozostały jedynie linie dot. bezproblemowego przebiegu eksperymentu.
czyli np. powinny zostać odrzucone takie linie jak
```
RUN 000039 COMPLETED. OUTPUT IN FILE yttrium.dat. 1 UNDERFLOW WARNING.
```
czy
```
RUN 000058 COMPLETED. OUTPUT IN FILE cerium.dat. ALGORITHM DID NOT CONVERGE AFTER 100000 ITERATIONS.
```
można jedynie założyć, że każda linijka pasuje do poniższego schematu
```
RUN [[NUMER Z DOKŁADNIE 6 CYFR]] COMPLETED. OUTPUT IN FILE [[NAZWA]].dat. [[COKOLWIEK]]
```

In [1]:
import re

pattern = r"^RUN \d{6} COMPLETED\. OUTPUT IN FILL [\w-]+\.dat\.\s$"

with open('atoms.log') as atoms:
    for line in atoms:
        if pattern.match(line):
            print(line)

FileNotFoundError: [Errno 2] No such file or directory: 'atoms.log'

### Zadanie 3 - Regexy

_Messages_

Napisz program który wypisze linie z logu systemowego "messages.txt" które zawierają informacje o nieprawidłowej nazwie użytkownika (invalid user)

In [2]:
pattern = re.compile(r"^(?P<month>\w{3}) (?P<day>\d{1,2}) (?P<hour>\d{2}):(?P<minute>\d{2}):(?P<second>\d{2}) [\d\w\-\[\]]+: Invalid user (?P<user>\w+) from (?P<ip>\d{1,3}(?:\.\d{1,3}){3})")

months = {
    "Jan": "stycznia",
    "Feb": "lutego",
    "Mar": "marca",
    "Apr": "kwietnia",
    "May": "maja",
    "Jun": "czerwca",
    "Jul": "lipca",
    "Aug": "sierpnia",
    "Sep": "września",
    "Nov": "pażdziernika",
    "Oct": "listopada",
    "Dec": "grudnia",
}


with open('messages.txt') as messages:
    for line in messages:
        if m := pattern.match(line):
           line = f"Niepoprawna nazwa użytkownika: {m.group('user')}, połączenie z {m.group('ip')} nawiązano {m.group('day')} {months.get(m.group('month'))} o godz. {m.group('hour')}"
           print(line)

FileNotFoundError: [Errno 2] No such file or directory: 'messages.txt'

Przerób program tak, żeby wyświetlał sformatowane komunikaty, tzn.
```
Jun 29 20:18:40 noether sshd[5805]: Invalid user tester from 218.189.194.200
```
powinno zostać przerobione na
```
Niepoprawna nazwa użytkownika: "tester", połaczenie z 218.189.194.200 nawiązano 29 czerwca o godz. 20:18
```

### Zadanie 4 - Collections
Uzupenij ponizsze funckje korzystajac z zaimporotwanych klas

In [3]:
from collections import defaultdict

def count_letters(word):
    """ Zwraca slownik gdzie kluczami sa litery, a wartosci ilosc ich wystepowania """
    dict = defaultdict(int)
    for letter in word:
       dict[letter] += 1
    return dict


print(count_letters('aaaaa') == {'a': 5})
print(count_letters('abbccaaaa') == {'a': 5, 'b': 2, 'c': 2})
print(count_letters('abc') == {'a': 1, 'b': 1, 'c': 1})

True
True
True


In [4]:
def group_words_by_first_letter(text):
    """ Dzieli tekst po spacjach i zwraca slownik gdzie kluczami sa pierwsze litery, a wartosciami listy slow zaczynajacych sie na te litery"""
    dict = defaultdict(list)
    text_list = text.split()
    for word in text_list:
        dict[word[0]].append(word)
    return dict

print(group_words_by_first_letter("ala ma kota") == {'a': ['ala'], 'm': ['ma'], 'k': ['kota']})
print(group_words_by_first_letter("ala ma kota ala ma psa") == {'a': ['ala', 'ala'], 'm': ['ma', 'ma'], 'k': ['kota'], 'p': ['psa']})   
print(group_words_by_first_letter("ala ma kota ale marysia ma konia") == {'a': ['ala', 'ale'], 'm': ['ma', 'marysia', 'ma'], 'k': ['kota', 'konia']})

True
True
True


### Zadanie 5 - Klasy
Uzupełnij brakujące metody poniższej klasy.

In [5]:
class FrozenDictionary(object):
    """
    Odpowiednik frozenset dla zbiorów, czyli słownik, który nie jest modyfikowalny,
    a dzięki temu może być np. elementem zbiorów, albo kluczem w innym słowniku.
    """
    
    def __init__(self, dictionary):
        """Tworzy nowy zamrożony słownik z podanego słownika"""
        self._dictionary = dictionary

    def __hash__(self):
        """Zwraca hasz słownika (int)"""
        return hash(tuple(sorted(self._dictionary.items())))


    def __eq__(self, d):
        """Porównuje nasz słownik z zamrożonym słownikiem d"""
        if not isinstance(d, FrozenDictionary):
            return False
        return self._dictionary == d._dictionary

    def __repr__(self):
        """Zwraca reprezentację naszego słownika jako string"""
        items = [f"{k} : {v}" for k, v in self._dictionary.items()]
        return "{" + ", ".join(items) + "}"

        
dicts = [FrozenDictionary({'ala': 4}), 
         FrozenDictionary({'ala': 1, 'jacek': 0}),
         FrozenDictionary({'ala': 4}), 
         FrozenDictionary({'ala': 2}), 
         FrozenDictionary({'jacek': 0, 'ala': 1})]

s = set(dicts)
print(dicts[0] == dicts[2])
print(dicts[0] != dicts[3])
print(len(s) == 3)
for d in dicts:
    print(d in s)

# Powinno wyświetlić coś w stylu set([{'ala': 4}, {'ala': 1, 'jacek': 0}, {'ala': 2}])
print(s)

True
True
True
True
True
True
True
True
{{ala : 1, jacek : 0}, {ala : 2}, {ala : 4}}


### Zadanie 6 - Bag of Words
Napisz klasę "Bag of words", która będzie "workiem słów" z zadanego dokumentu. Oznacza to, że ma w sobie przechowywać informacje o tym ile razy każde słowo wystąpiło w dokumencie. Dodatkowo ma udostępniać następującą funkcjonalność:
* Można utworzyć go na dwa sposoby:
```
bow = BagOfWords("ala ma kota ala ma ala")
bow = BagOfWords(open("plik.txt"))
```
* Możliwość wyświetlania go po prostu pisząc print(bow) (kolejność taka sama jak przy przeglądaniu forem)
```
ala:3, ma:2, kota:1
```
* Sprawdzanie czy jakieś słowo należy do naszego worka pisząc
```
if 'ala' in bow:
    ...
```
* Przeglądanie słów w worku od najczęściej do najrzadziej występującego, czyli
```
for word in bow:
  print(word)
```
    powinniśmy zobaczyć
```
ala
ma
kota
```
* Możliwość dodawania dwóch worków, pisząc
```
bow1 = BagOfWords("ala ma kota ala ma ala")
bow2 = BagOfWords("tomek tez ma kota")
bow3 = bow1 + bow2
print('tomek' in bow1) # False
print('tomek' in bow3) # True
print('ala' in bow3) # True
print(bow3) # ala:3, ma:3, kota:2, tez:1, tomek:1
    ```
* Odczytywanie częstości wystąpień słów poprzez nawiasy kwadratowe
```
print(bow1["ala"]) # 3
print(bow3["ala"]) # 3
```
* Aktualizację cześtości wystąpień
```
bow3['tomek'] = 10
for el in bow3:
    print el
```
    powinno wyświetlić najpierw `tomek`

In [6]:
from collections import Counter

class BagOfWords:

    def __init__(self, words):
        if hasattr(words, "read"):
            words = words.read()
        self._word_dict = Counter(words.split())


    def __repr__(self):
        s = ""
        i = 0
        for k, v in self._word_dict.items():
            if i != 0:
                s += ", "
            s += f"{k}:{v}"
            i += 1
        return s

    def __contains__(self, word):
        if self._word_dict.get(word, 0) > 0:
            return True
        return False

    def __iter__(self):
        return (word for word, count in self._word_dict.most_common())

    def __add__(self, other):
        bag = BagOfWords("")
        bag._word_dict = self._word_dict + other._word_dict
        return bag

    def __getitem__(self, item):
        return self._word_dict[item]

    def __setitem__(self, item, value):
        self._word_dict[item] = value


# Przykładowe testy demonstracyjne dla BagOfWords

bow1 = BagOfWords("ala ma kota ala ma ala")
bow2 = BagOfWords("tomek tez ma kota")
bow3 = bow1 + bow2

# Test 1: dostęp do liczb wystąpień
print((bow1["ala"] == 3) == True)
print((bow1["ma"] == 2) == True)
print((bow1["kota"] == 1) == True)

# Test 2: sprawdzanie obecności słów
print(("tomek" in bow1) == False)
print(("tomek" in bow3) == True)
print(("ala" in bow3) == True)

# Test 3: iteracja po słowach (od najczęstszych)
expected_words = ["ala", "ma", "kota"]
print(list(bow1) == expected_words)

# Test 4: dodawanie worków
print(str(bow3) == "ala:3, ma:3, kota:2, tomek:1, tez:1")

# Test 5: aktualizacja liczby wystąpień
bow3["tomek"] = 10
expected_order = ["tomek", "ala", "ma", "kota", "tez"]
print(list(bow3) == expected_order)

# Test 6: porównania i proste operacje logiczne
dicts = [bow1._word_dict, bow2._word_dict, bow3._word_dict]  # używamy Counter wewnętrznie
print((dicts[0] == dicts[2]) == False)
print((dicts[0] != dicts[1]) == True)


True
True
True
True
True
True
True
True
True
True
True
